# Note

I made this notebook in hurry and I believe there is alot of room for improvement.

***Data preparation: [Data Prepare - Separate Spectogram](https://www.kaggle.com/code/muhammad4hmed/hms-data-prepare-separate-spectogram)***

***Training Notebook: [Train - ResNet18 on Spectrograms](https://www.kaggle.com/code/muhammad4hmed/hms-train-spectogram-images)***

> Upvote if you like the approach!

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import pandas as pd
from PIL import Image
import torchvision.models as models
from tqdm import tqdm
from torch.nn.functional import softmax, one_hot, log_softmax
from io import BytesIO
import matplotlib.pyplot as plt
from transformers import ViTModel
import numpy as np
import matplotlib.cm as cm
from PIL import Image

In [ ]:
train = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
test = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
classes = train['expert_consensus'].unique()
mapping = {
    c:i for i, c in enumerate(classes)
}
num_classes = classes.shape[0]

In [ ]:
cmap = cm.get_cmap("viridis")

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        specto_id = self.data.loc[idx, 'spectrogram_id']
        specto_path = f'/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/{specto_id}.parquet'
        specto = pd.read_parquet(specto_path)
        spectrogram = Image.fromarray((cmap(specto) * 255).astype(np.uint8))
        if self.transform:
            spectrogram = self.transform(spectrogram)[:3, :, :]
        return spectrogram

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
#     transforms.CenterCrop(224),  # Center crop to maintain aspect ratio
    transforms.ToTensor(),  # Convert to PyTorch tensor
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize (optional)
])

In [ ]:
test_dataset = ImageDataset(test, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class ViTClassifier(torch.nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        self.vit = ViTModel.from_pretrained("/kaggle/input/google-vit-base-patch16-224-in21k")
        self.classifier = torch.nn.Linear(self.vit.config.hidden_size, num_classes)

    def forward(self, images):
        output = self.vit(images)
        output = self.classifier(output.last_hidden_state[:, 0]) 
        output = softmax(output, dim = 1)
        return output

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ViTClassifier(num_classes).to(device)
model.load_state_dict(torch.load('/kaggle/input/hms-train-spectogram-images/trained_model.pt'))

In [ ]:
model.eval()
out = []
pbar = tqdm(test_loader)
for images in pbar:
    images = images.to(device)
    with torch.no_grad():
        outputs = model(images)
    outputs = outputs.detach().cpu().numpy()
    out.append(outputs)

In [ ]:
outputs = np.vstack(out)

In [ ]:
mapping

In [ ]:
submission = test[['eeg_id']]
submission['seizure_vote'] = outputs[:, 0]
submission['lpd_vote'] = outputs[:, 5]
submission['gpd_vote'] = outputs[:, 1]
submission['lrda_vote'] = outputs[:, 2]
submission['grda_vote'] = outputs[:, 4]
submission['other_vote'] = outputs[:, 3]

In [ ]:
submission.to_csv('submission.csv', index =- False)
submission